In [1]:
import json

validation_samples={}
def get_samples_from_validation_scenes(nusc, validation):

    for scene in nusc.scene:  # 모든 Scene을 순회
        scene_token = scene['token']
        scene_name = scene['name']  # 'scene-0061'

        if scene_name in validation:
            current_token = scene['first_sample_token']
            samples = []
            while current_token:
                sample = nusc.get('sample', current_token)
                samples.append(sample['token'])
                current_token = sample['next']  # 다음 Sample로 이동

            validation_samples[scene_name] = samples
    return validation_samples

In [2]:
import os

def get_image_paths_for_scene(samples, visualize_folder):
    image_paths = []
    for token in samples:
        image_path = os.path.join(visualize_folder, f"{token}.jpg")
        if os.path.exists(image_path):
            image_paths.append(image_path)
        else:
            print(f"Image not found: {image_path}")
    return image_paths

In [3]:
import cv2

def create_video_from_images(image_paths, output_path, fps=20):
    if not image_paths:
        print("No images to create video!")
        return

    # 첫 번째 이미지로 비디오 크기 설정
    first_image = cv2.imread(image_paths[0])
    height, width, _ = first_image.shape

    # 비디오 파일 설정
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # MP4 파일
    video = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # 이미지 추가
    for img_path in image_paths:
        img = cv2.imread(img_path)
        if img is None:
            print(f"Error loading image: {img_path}")
            continue
        video.write(img)

    video.release()
    print(f"Video saved at {output_path}")

In [4]:
# NuScenes 데이터셋 로드
from nuscenes.nuscenes import NuScenes

nusc = NuScenes(version='v1.0-mini', dataroot='./data/nuscenes/v1.0-mini', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.7 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [5]:
from nuscenes.utils.splits import create_splits_scenes

visualize_folder = "output/home/ircv-agx/sl/DSVT/tools/cfgs/dsvt_models/dsvt_plain_1f_onestage_nusences/default/eval/epoch_no_number/val/default/final_result/data/visualize"
output_folder = "./video"
validation_scenes = create_splits_scenes(verbose=False)['val']

os.makedirs(output_folder, exist_ok=True)

validation_samples = get_samples_from_validation_scenes(nusc, validation_scenes)

In [6]:

for scene_name, samples in validation_samples.items():
    print(f"Processing scene: {scene_name}")

    image_paths = get_image_paths_for_scene(samples, visualize_folder)

    video_path = os.path.join(output_folder, f"{scene_name}.mp4")

    create_video_from_images(image_paths, video_path, fps=2)

Processing scene: scene-0103
Video saved at ./video/scene-0103.mp4
Processing scene: scene-0553
Image not found: output/home/ircv-agx/sl/DSVT/tools/cfgs/dsvt_models/dsvt_plain_1f_onestage_nusences/default/eval/epoch_no_number/val/default/final_result/data/visualize/8687ba92abd3406aa797115b874ebeba.jpg
Image not found: output/home/ircv-agx/sl/DSVT/tools/cfgs/dsvt_models/dsvt_plain_1f_onestage_nusences/default/eval/epoch_no_number/val/default/final_result/data/visualize/5bbeb4e68d51413f81a370cb33ef27fb.jpg
Image not found: output/home/ircv-agx/sl/DSVT/tools/cfgs/dsvt_models/dsvt_plain_1f_onestage_nusences/default/eval/epoch_no_number/val/default/final_result/data/visualize/e9a1c915c0d641d0b0a3019112a1ee90.jpg
Image not found: output/home/ircv-agx/sl/DSVT/tools/cfgs/dsvt_models/dsvt_plain_1f_onestage_nusences/default/eval/epoch_no_number/val/default/final_result/data/visualize/f5b8088835f247b8a5434a9f0daadc20.jpg
Image not found: output/home/ircv-agx/sl/DSVT/tools/cfgs/dsvt_models/dsvt_pl